In [76]:
import gym
import sconegym
import sys
sys.path.append("C:/Program Files/SCONE/bin")
import yaml
import deprl
import myutils
from myutils.load_utils import load
from myutils import get_directory_path, inference,make6
import os
import datetime
import numpy as np

In [77]:
name = "fix_stiffness_new_"
name = "fix_stiffness_new_composite_"
name = "fix_stiffness_new_composite_ori_"
# name = "fix_stif_10_"
name = "fix_reward" 
name = "fix_10_default"

In [78]:
dir_path  = get_directory_path() # result directory
with open(f'trainconfig/{name}.yaml','r') as f:
    config = yaml.safe_load(f)
path = os.path.join(dir_path,name)
names = os.listdir(path)[0]
path = os.path.join(path,names)

### best case

In [79]:
import pandas as pd
log = pd.read_csv(os.path.join(path,'log.csv'))

In [80]:
def per_5(df):
    result = pd.DataFrame()
    for i in range(len(df)//5):
        temp = df[i*5:(i+1)*5].mean()
        result = pd.concat([result,pd.DataFrame([temp])])
    return result

In [85]:
def find_max_mean(df):
    df = per_5(df)
    df = df.reset_index()
    # column = 'train/episode_score/mean'
    column = 'train/episode_score/mean'
    max_mean = df[column].max()
    max_mean_index = np.where(df[column]==max_mean)[0][-1]
    print(f"The maximum mean value of the test episodes is {round(max_mean,2)} at index {max_mean_index*5}.")
    return str(int(round(max_mean_index+1,0)*1e6))
    
# Example usage:
ch = find_max_mean(log)

The maximum mean value of the test episodes is 2788.95 at index 155.


In [86]:
max_mean_index = 250/5
ch = str(int(round(max_mean_index+1,0)*1e6))

### Simul

In [87]:
env = eval(config['tonic']['environment'])
time = datetime.datetime.now() 
env.set_output_dir(config['tonic']['name']+f'{make6(time)}')
policy = deprl.load(os.path.join(path,'checkpoints'),env,checkpoint=ch)  ## checkpoint =  ch로 설정하면 최상의 결과
print(env.output_dir)
pos, vel = inference(env,policy,num=5,save = True,Max = True,more=False,reward_type = 'position',types = 'both')

Loading
fix_10_default222219
Episode 1: steps=1000; reward=8981.112, com=[ 12.023200 0.967432 0.124518 ]
Episode 2: steps=1000; reward=8975.889, com=[ 12.185416 0.968219 0.029468 ]
Episode 3: steps=1000; reward=8989.883, com=[ 12.032866 0.972439 0.012728 ]
Episode 4: steps=133; reward=999.164, com=[ 1.103518 0.586266 0.090969 ]
Episode 5: steps=109; reward=819.205, com=[ 0.898737 0.570089 -0.175022 ]
Epis: 3,Max_Re:8989.883062242643



###